In [ ]:
!pip install pandas dask psutil kaggle --quiet


In [ ]:
from google.colab import files
import os

uploaded = files.upload()

os.makedirs(os.path.expanduser("~/.kaggle"), exist_ok=True)

for filename in uploaded.keys():
    os.rename(filename, os.path.expanduser("~/.kaggle/kaggle.json"))

os.chmod(os.path.expanduser("~/.kaggle/kaggle.json"), 0o600)

Saving kaggle.json to kaggle.json


In [ ]:
import os, zipfile


!kaggle datasets download -d mkechinov/ecommerce-behavior-data-from-multi-category-store


with zipfile.ZipFile("ecommerce-behavior-data-from-multi-category-store.zip", "r") as z:
    z.extractall(".")

!ls -lh
print("الملفات الموجودة:", os.listdir("."))

Dataset URL: https://www.kaggle.com/datasets/mkechinov/ecommerce-behavior-data-from-multi-category-store
License(s): copyright-authors
100% 4.28G/4.29G [01:48<00:00, 59.9MB/s]
100% 4.29G/4.29G [01:48<00:00, 42.4MB/s]
total 18G
-rw-r--r-- 1 root root 8.4G Oct 22 19:50 2019-Nov.csv
-rw-r--r-- 1 root root 5.3G Oct 22 19:51 2019-Oct.csv
-rw-r--r-- 1 root root 4.3G Dec  9  2019 ecommerce-behavior-data-from-multi-category-store.zip
drwxr-xr-x 1 root root 4.0K Oct 21 16:52 sample_data
الملفات الموجودة: ['.config', '2019-Nov.csv', 'ecommerce-behavior-data-from-multi-category-store.zip', '2019-Oct.csv', 'sample_data']


In [ ]:
import pandas as pd

csv_file = "/content/2019-Nov.csv"

sample_df = pd.read_csv(csv_file, nrows=5)
print("معاينة أولية للبيانات:")
print(sample_df.head())

معاينة أولية للبيانات:
                event_time event_type  product_id          category_id  \
0  2019-11-01 00:00:00 UTC       view     1003461  2053013555631882655   
1  2019-11-01 00:00:00 UTC       view     5000088  2053013566100866035   
2  2019-11-01 00:00:01 UTC       view    17302664  2053013553853497655   
3  2019-11-01 00:00:01 UTC       view     3601530  2053013563810775923   
4  2019-11-01 00:00:01 UTC       view     1004775  2053013555631882655   

               category_code   brand   price    user_id  \
0     electronics.smartphone  xiaomi  489.07  520088904   
1  appliances.sewing_machine  janome  293.65  530496790   
2                        NaN   creed   28.31  561587266   
3  appliances.kitchen.washer      lg  712.87  518085591   
4     electronics.smartphone  xiaomi  183.27  558856683   

                           user_session  
0  4d3b30da-a5e4-49df-b1a8-ba5943f1dd33  
1  8e5f4f83-366c-4f70-860e-ca7417414283  
2  755422e7-9040-477b-9bd2-6a6e8fd97387  
3  3bfb58

In [ ]:
import time
import psutil
import pandas as pd
import os

start_time = time.time()

chunk_size = 5000
chunks = []
process = psutil.Process(os.getpid())


for i, chunk in enumerate(pd.read_csv(csv_file, chunksize=chunk_size)):
    chunks.append(len(chunk))
    print(f" دفعة {i+1}: {len(chunk)} صفوف ، الذاكرة الحالية: {process.memory_info().rss / 1024 ** 2:.2f} MB")

total_rows = sum(chunks)
elapsed_chunks = time.time() - start_time
memory_chunks = process.memory_info().rss / 1024 ** 2


print(f"عدد الصفوف الكلي: {total_rows}")
print(f"الوقت المستغرق: {elapsed_chunks:.2f} ثانية باستخدام chunksize")

Streaming output truncated to the last 5000 lines.
 دفعة 8504: 5000 صفوف ، الذاكرة الحالية: 138.82 MB
 دفعة 8505: 5000 صفوف ، الذاكرة الحالية: 138.82 MB
 دفعة 8506: 5000 صفوف ، الذاكرة الحالية: 138.82 MB
 دفعة 8507: 5000 صفوف ، الذاكرة الحالية: 138.82 MB
 دفعة 8508: 5000 صفوف ، الذاكرة الحالية: 138.82 MB
 دفعة 8509: 5000 صفوف ، الذاكرة الحالية: 138.82 MB
 دفعة 8510: 5000 صفوف ، الذاكرة الحالية: 138.82 MB
 دفعة 8511: 5000 صفوف ، الذاكرة الحالية: 138.82 MB
 دفعة 8512: 5000 صفوف ، الذاكرة الحالية: 138.82 MB
 دفعة 8513: 5000 صفوف ، الذاكرة الحالية: 138.82 MB
 دفعة 8514: 5000 صفوف ، الذاكرة الحالية: 138.82 MB
 دفعة 8515: 5000 صفوف ، الذاكرة الحالية: 138.82 MB
 دفعة 8516: 5000 صفوف ، الذاكرة الحالية: 138.82 MB
 دفعة 8517: 5000 صفوف ، الذاكرة الحالية: 138.82 MB
 دفعة 8518: 5000 صفوف ، الذاكرة الحالية: 138.82 MB
 دفعة 8519: 5000 صفوف ، الذاكرة الحالية: 138.83 MB
 دفعة 8520: 5000 صفوف ، الذاكرة الحالية: 138.83 MB
 دفعة 8521: 5000 صفوف ، الذاكرة الحالية: 138.83 MB
 دفعة 8522: 5000 صفوف ، الذاكرة

In [ ]:
!pip install dask

import dask.dataframe as dd
import time
import psutil
import os

start_time = time.time()

df_dask = dd.read_csv(csv_file)

row_count = df_dask.shape[0].compute()

elapsed_dask = time.time() - start_time

process = psutil.Process(os.getpid())
memory_dask = process.memory_info().rss / 1024 ** 2


print(f"عدد الصفوف: {row_count}")
print(f"الوقت المستغرق بـ Dask: {elapsed_dask:.2f} ثانية")
print(f"الذاكرة الحالية بـ Dask: {memory_dask:.2f} MB")

عدد الصفوف: 67501979
الوقت المستغرق بـ Dask: 187.33 ثانية
الذاكرة الحالية بـ Dask: 572.27 MB


In [ ]:
import gzip
import shutil
import time

start_compress = time.time()
with open(csv_file, 'rb') as f_in:
    with gzip.open('data_compressed.csv.gz', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)
compress_time = time.time() - start_compress

start_read = time.time()
df_compressed = pd.read_csv('data_compressed.csv.gz', compression='gzip', nrows=50000)
read_time = time.time() - start_read
memory_compressed = process.memory_info().rss / 1024 ** 2

print(f" وقت الضغط: {compress_time:.2f} ثانية")
print(f" وقت القراءة بعد الضغط: {read_time:.2f} ثانية")
print(f"الذاكرة:: {memory_compressed:.2f}  MB")
print(f" حجم الملف قبل وبعد الضغط:")
!ls -lh *.csv*


 وقت الضغط: 794.88 ثانية
 وقت القراءة بعد الضغط: 0.59 ثانية
الذاكرة:: 589.64  MB
 حجم الملف قبل وبعد الضغط:
-rw-r--r-- 1 root root 8.4G Oct 22 19:50 2019-Nov.csv
-rw-r--r-- 1 root root 5.3G Oct 22 19:51 2019-Oct.csv
-rw-r--r-- 1 root root 2.7G Oct 22 20:12 data_compressed.csv.gz


In [ ]:
import pandas as pd
import psutil
import os

memory_compressed = psutil.Process(os.getpid()).memory_info().rss / 1024 ** 2


results = pd.DataFrame({
    'الطريقة': ['Pandas + chunksize', 'Dask', 'Pandas + Compression'],
    'الزمن التقريبي (ثواني)': [elapsed_chunks, elapsed_dask, compress_time],
    'الذاكرة (MB)': [memory_chunks, memory_dask, memory_compressed],

})

results

,الطريقة,الزمن التقريبي (ثواني),الذاكرة (MB)
0,Pandas + chunksize,229.615716,112.980469
1,Dask,187.328720,572.273438
2,Pandas + Compression,794.879133,590.386719



#  تقرير حول مقارنة ثلاث طرق لمعالجة البيانات الضخمة

## 1. المقدمة

يهدف هذا التقرير إلى دراسة ومقارنة ثلاث خوارزميات أو أساليب مختلفة لمعالجة ملفات البيانات الكبيرة (ملفات CSV) باستخدام مكتبات Python الشائعة، وهي:

* **Pandas مع chunksize** (القراءة المجزأة).
* **Dask** (المعالجة المتوازية والموزعة).
* **Pandas مع ضغط البيانات (Compression)**.

تم إجراء التجارب على بيانات من متجر إلكتروني (ecommerce-behavior-data) تم تنزيلها من Kaggle.
لكل طريقة، تم قياس **الزمن المستغرق** و**استهلاك الذاكرة** أثناء القراءة أو المعالجة.


## 2. الطريقة الأولى: Pandas + chunksize


تستخدم هذه الطريقة ميزة `chunksize` في مكتبة **Pandas**، بحيث يتم تحميل الملف على دفعات صغيرة (Chunks) من البيانات بدلاً من تحميله بالكامل في الذاكرة دفعة واحدة.
في هذا المشروع، تم تحديد حجم الدفعة بـ 5000 صف.

### الإيجابيات

*  تقلل من استهلاك الذاكرة عند التعامل مع ملفات ضخمة.
*  بسيطة وسهلة التنفيذ باستخدام أدوات Pandas المعروفة.
*  تسمح بمعالجة الدفعات واحدة تلو الأخرى دون الحاجة إلى موارد ضخمة.

### السلبيات

*  المعالجة تكون **تسلسلية** وليست متوازية، مما قد يجعلها أبطأ عند توفر أنوية CPU متعددة.
*  دمج نتائج الدفعات لاحقًا قد يتطلب جهدًا إضافيًا.
*  اختيار حجم chunk غير مناسب قد يؤدي إلى بطء في التنفيذ أو زيادة في الذاكرة.

### الخلاصة

مناسبة للبيانات الكبيرة التي لا يمكن تحميلها دفعة واحدة في الذاكرة، مع بساطة في التنفيذ وكفاءة جيدة في استهلاك الذاكرة، لكنها أبطأ مقارنة بالطرق المتوازية.



## 3. الطريقة الثانية: Dask

### الوصف

تعتمد هذه الطريقة على مكتبة **Dask**، التي تسمح بالتعامل مع البيانات الكبيرة بطريقة متوازية (Parallel Processing) مع الحفاظ على واجهة تشبه Pandas.
تم استخدام `dd.read_csv()` لتحميل البيانات ومعالجتها عبر عدة أجزاء (Partitions) بشكل متوازي.

### الإيجابيات

*  قادرة على التعامل مع بيانات تفوق حجم الذاكرة RAM.
*  تستفيد من تعدد الأنوية وتنفذ العمليات بشكل متوازي.
*  توفر قابلية التوسع إلى بيئات موزعة (Cluster Computing).
*  واجهتها قريبة من Pandas، مما يسهل الانتقال إليها.

### السلبيات

*  إعدادها أكثر تعقيدًا مقارنة بـ Pandas.
*  تحتاج إلى ضبط حجم الـ Partitions لتحقيق الأداء الأمثل.
*  بعض وظائف Pandas غير مدعومة أو بطيئة في Dask.
*  عند العمل على بيانات صغيرة، قد يكون أداء Dask أبطأ من Pandas بسبب الـ overhead.

### الخلاصة

Dask خيار ممتاز للبيانات الكبيرة جدًا وللمهام التحليلية المتوازية، خاصة في الأنظمة متعددة الأنوية أو البيئات الموزعة.


## 4. الطريقة الثالثة: Pandas + Compression (gzip)

### الوصف

تعتمد هذه الطريقة على ضغط البيانات لتقليل حجم الملف باستخدام صيغة **gzip**.
تم ضغط الملف أولاً ثم قراءته بواسطة `pd.read_csv(..., compression='gzip')`.

### الإيجابيات

*  تقليل حجم الملف بشكل كبير، مما يوفر في مساحة التخزين والنقل.
*  في بعض الحالات، تكون القراءة أسرع لأن كمية البيانات المقروءة من القرص أقل.
*  مدعومة بشكل مباشر من Pandas بدون إعدادات إضافية.

### السلبيات

*  عملية الضغط تستغرق وقتا إضافيا قبل القراءة.
*  gzip لا يدعم القراءة المتوازية أو الوصول العشوائي للبيانات بسهولة.
*  لا يناسب المعالجة المتكررة (لأن كل مرة تحتاج لفك الضغط).
*  أبطأ في الاستعلامات الجزئية مقارنة بتنسيقات حديثة مثل Parquet.

### الخلاصة

طريقة فعالة لتقليل الحجم عند أرشفة أو نقل البيانات، لكنها ليست الخيار الأفضل عند الحاجة لمعالجة تفاعلية أو متكررة.


## 5. مقارنة عامة بين الطرق الثلاث

| الطريقة                  | الزمن المستغرق                      | الذاكرة       | سهولة الاستخدام  | قابلية التوسع  | التعليق العام                     |
| ------------------------ | -------------------------------------- | --------------- | ------------------ | ---------------- | --------------------------------- |
| **Pandas + chunksize**   | متوسط                                  | منخفض           | سهل جدا           | محدود            | مناسب للمعالجة المتسلسلة البسيطة  |
| **Dask**                 | سريع جدا (مع بيانات ضخمة)             | متوسط إلى مرتفع | متوسط              | عالي جدا        | الأفضل للأداء والمعالجة المتوازية |
| **Pandas + Compression** | بطيء (عند الضغط) / متوسط (عند القراءة) | منخفض           | سهل                | محدود            | جيد للتخزين والنقل فقط            |




## 6. الخاتمة

من خلال المقارنة، يتبين أن:

* طريقة **Pandas + chunksize** هي الأبسط والأكثر اقتصادية في الذاكرة.
* طريقة **Dask** هي الأقوى أداء والأكثر قابلية للتوسع.
* طريقة **Pandas + Compression** مفيدة لأغراض التخزين وليس التحليل السريع.

اختيار الطريقة المثلى يعتمد على **حجم البيانات، نوع المهمة، والموارد المتاحة**.

